In [22]:
from hashlib import sha256
import random

class Digital_Signature:
    sender = "unknown"
    receiver = "unknown"
    
    def __init__(self):
        print("=== Digital Signature ===")
        self.sender = "Jack"
        self.receiver = "Mary"
        
        print("Sender: " + self.sender + " | Receiver: " + self.receiver, end="\n\n")
    
    def show_menu(self):
        # Clearing the Screen
        print("Operations: ", end = "\n")
        print("\tG - Generate Primes :: ", end = "\n")
        print("\tK - Generate Keys :: ", end = "\n")
        print("\tS - Sign a text :: ", end = "\n")
        print("\tV - Verify signature :: ", end = "\n")
        print("\tE - Exit :: ", end = "\n")
        
        print("Please, enter the operation: ", end = "\n")
        option = input()
        option = option.strip()
        
        print(end = "\n")
        
        if option == "G" or option == 'g':
            self.show_generate_primes()
        elif option == "K" or option == 'k':
            self.show_generate_keys()
        elif option == "S" or option == 's':
            self.show_sign_text()
        elif option == "V" or option == 'v':
            self.show_verify_signature()
        elif option == "E" or option == 'e':
            self.dismiss()
        else:
            print("Invalid option. Please, enter a sigle letter according to the desired operation!", end = "\n")
            self.show_menu()
        
    def generate_primes(self):
        upper_bound = 2**(256) - 1
        lower_bound = 2**(254) - 1
        
        p = random_prime(upper_bound, False, lower_bound)
        q = random_prime(upper_bound, False, lower_bound)
        while(p == q):
            q = random_prime(upper_bound, False, lower_bound)
        return p, q
        
    def show_generate_primes(self):
        p, q = self.generate_primes()
        
        """
        print("Enter the text to be signed: ")
        text = input()
        print("Enter the private key: ")
        private_key = input()
        """
        
        print(f'First prime (p): {p}' , end="\n")
        print(f'Second prime (q): {q}', end="\n")
        print(end="\n")
        
        self.closeOrNot()    
        
    def show_generate_keys(self):
        print("Enter the first prime(p): ")
        p = Integer(input())
        print("Enter the second prime(q): ")
        q = Integer(input())
        
        public_key, private_key = self.rsa_key_generation(p, q)
        
        print(end="\n")
        print(end="\n")
        print(f'Public key: {public_key}', end = "\n")
        print(end="\n")
        print(f'Private key: {private_key}', end = "\n")
        
        print(end="\n")
        
        self.closeOrNot()
    
    def show_sign_text(self):
        print("Enter the text to be signed: ")
        text = input().strip()
        
        print("Enter the private key (p): ")
        e = Integer(input().strip())
        
        print("Enter the private key (n): ")
        n = Integer(input().strip())
        
        private_key = e, n
        
        signature = self.sign_text(text, private_key)
        
        print("Signature of the given text: " + str(signature), end = "\n")
        print(end="\n")
        
        self.closeOrNot()
        
    
    def show_verify_signature(self):
        print("Enter the text to be verified: ")
        text = input().strip()
        
        print("Enter the corresponding signature to check: ")
        signature = input().strip()
        
        print("Enter the public key (d) of the given pair: ")
        d = Integer(input().strip())
        
        print("Enter the public key (n) of the given pair: ")
        n = Integer(input().strip())
        
        public_key = d, n
        
        print("\nCkecking ... ", end = '')
        
        load = int(0)
        while load <= 100:
            print(f'{load}%', end = ' ')
            rand = int(random.randint(0, 100))
            sleep(round((rand+1)/24))
            load += rand
        
        print("100%", end="\n\n")
        
        print("Verdict: ", end = '')
        
        res = self.verify_signature(text, signature, public_key)
        if res == True:
            print("Sucess! The signature is valid.", end = "\n")
        else:
            print("Fail! Invalid signature.", end = "\n")
        
        
        print(end="\n")
        self.closeOrNot()
        
    #def show_verify_signature(self):
     #   print("Generate Primes: ", end = "\n")
    
    def dismiss(self):
        print(end="\n")
        print("THANK YOU FOR USING MY SCRIPT :)")
        print("Copyright© Application of Discrete Models ELTE 2022, Alfredo Martins (Student) & Dr. Mohammed B. M. Kamel (Teacher)")
        quit()
              
    def closeOrNot(self):
        print("Would you like to close the program? Enter 'Y' to close", end = "\n")
        option = input().strip()
              
        if option == "Y" or option == "y":
            self.dismiss()
        else:
            sleep(1)
            self.show_menu()

    def rsa_key_generation(self, p, q):

        n = p*q
        phi = (p-1)*(q-1) #euler_phi(n) is slower

        e = ZZ.random_element(phi) # [2, n - 1] -> Good
        while gcd(phi, e) != 1 or e < 2:
            e = ZZ.random_element(phi)

       # By applying Bézout theorem, we get the coefficients 
        bezout_coefficient = xgcd(e, phi)
        d = Integer(mod(bezout_coefficient[1], phi)) # Inverse_mod function <--- ???

        mod_result = mod(d*e, phi)
        if mod_result == 1: #We are good to go
            public_key = d, n # (n, e)
            private_key = e, n # (n, d)

            return public_key, private_key # Keys generated
        return None
    
    def bin_pow(self, a, b, m):
        a = mod(a, m)
        res = Integer(1)
        while b > 0:
            if b & 1:
                res = mod(res * a, m)
            a = mod(a * a, m);
            b >>= 1
        return res


    def rsa_encryption(self, m, private_key):
        e, n = private_key
        
        #c = power_mod(m, e, n)
        c = self.bin_pow(m, e, n)

        #print(f"Hash: {m}", end = "\n")
        #print(f"Encrypted: {c}", end = "\n")

        return c
    
    def get_char_representation(self, text):
        list_text = []
        for c in text:
            list_text.append(ord(c))
                
        final_value = Integer()
        for x in list_text:
            final_value = final_value*100 + x
        
        return final_value

    def rsa_decryption(self, c, public_key):
        d, n = public_key
        
        c = Integer(mod(c, n))
        n = Integer(n)
        d = Integer(d)
        #m = power_mod(c, d, n)
        m = self.bin_pow(c, d, n)
        
        #print(f"Hash: {c}", end = "\n")
        #print(f"Decrypted: {m}", end = "\n")
        
        return m
    
    def sign_text(self, plain_text, private):
        m = self.get_hash_text(plain_text)
        signature = self.rsa_encryption(m, private)
        return signature
        
    def get_hash_text(self, text):      
        m = int.from_bytes(sha256(text.encode()).digest(), byteorder='big')
        return m
        
    def verify_signature(self, plain_text, signature, public_key):
        text_hash = self.get_hash_text(plain_text)
        
        original_text_hash = self.rsa_decryption(signature, public_key)
        
        #print(f"Decrypted: {original_text_hash}", end = "\n")
        #print(f"Decrypted hash functon: {self.get_hash_text(str(original_text_hash))}", end = "\n")
        #print(f"Hashed text: {text_hash}", end = "\n")
        
        if original_text_hash == text_hash:
            return True
        return False

In [25]:
obj = Digital_Signature()
obj.show_menu()

=== Digital Signature ===
Sender: Jack | Receiver: Mary

Operations: 
	G - Generate Primes :: 
	K - Generate Keys :: 
	S - Sign a text :: 
	V - Verify signature :: 
	E - Exit :: 
Please, enter the operation: 
v

Enter the text to be verified: 
bi
Enter the corresponding signature to check: 
5
Enter the public key (d) of the given pair: 
5
Enter the public key (n) of the given pair: 
15

Ckecking ... 0% 70% 74% 100%

Verdict: Fail! Invalid signature.

Would you like to close the program? Enter 'Y' to close
y

THANK YOU FOR USING MY SCRIPT :)
Copyright© Application of Discrete Models ELTE 2022, Alfredo Martins (Student) & Dr. Mohammed B. M. Kamel (Teacher)


In [24]:
obj = Digital_Signature()
#obj.show_menu()
text = "HELLOWORLD"
res = obj.get_char_representation(text)
print(res)

=== Digital Signature ===
Sender: Jack | Receiver: Mary

72697676798779827668


In [ ]:
obj = Digital_Signature() m = 72697676798779827668 e = 1850567623300615966303954877 n = 4951760154835678088235319297

signature = obj.rsa_encryption(m, (e, n))

c = power_mod(m, e, n) print(c)

c = 630913632577520058415521090 d = 4460824882019967172592779313 n = 4951760154835678088235319297

res = obj.rsa_decryption(c, (d, n))

power_mod(c, d, n)